In [1]:
import json # build-in library
import arff # https://pypi.python.org/pypi/liac-arff

In [2]:
features = [("word count", 'NUMERIC'),
            ("average word length", 'NUMERIC'),
            ("length of the longest word", 'NUMERIC'),
            ("whether start with number", ['True', 'False']),
            ("whether start with who/what/why/where/when/how", ['True', 'False']),
            ("label", ['0', '1'])]

In [3]:
def extract_features(text):
    doc = text.strip().split(' ')
    f1 = 0
    f2 = 0
    f3 = 0
    f4 = False
    f5 = False
    for token in doc:
        word = token.lower()
        if f1 == 0:
            if word[0].isdigit():
                f4 = True
            if word in ['who', 'what', 'why', 'where', 'when', 'how']:
                f5 = True
        f1 += 1
        length = len(word)
        f2 += length
        f3 = max(f3, length)
    if f1 == 0:
        return False
    return (f1, f2 * 1.0 / f1, f3, f4, f5)

In [14]:
id_features = {}
with open('../clickbait_dataset/instances_train.jsonl', 'r') as f:
    for line in f:
        dic = json.loads(line)
        if len(dic['postText'][0]) > 0:
            feat = extract_features(dic['postText'][0])
            if feat != False:
                id_features.setdefault(dic['id'], feat)
        else:
            print(dic['id'])
            print(dic['postText'])
            
print(len(id_features))

2196
['']
2213
['']
2217
['']
2235
['']
2238
['']
2272
['']
3781
['']
3824
['']
3830
['']
3865
['']
5292
['']
5322
['']
5347
['']
5372
['']
5381
['']
6766
['']
6829
['']
6835
['']
6855
['']
6882
['']
6926
['']
8424
['']
8474
['']
8539
['']
8904
['']
9925
['']
9941
['']
9981
['']
11527
['']
11635
['']
13163
['']
13171
['']
13204
['']
13206
['']
13233
['']
14695
['']
14704
['']
14750
['']
14760
['']
14780
['']
16251
['']
16280
['']
16288
['']
16300
['']
16302
['']
16306
['']
17535


In [18]:
id_labels = []
with open('../clickbait_dataset/truth_train.jsonl', 'r') as f:
    for line in f:
        dic = json.loads(line)
        label = 1
        if dic['truthClass'][0] == 'n':
            label = 0
        if dic['id'] in id_features:
            id_labels.append(label)
#print((id_labels))


In [16]:
#truthClass non-clickbait = 0
#clickbait = 1
print(id_labels[1976])

1


In [6]:
###### dump data into arff file

data = {}
data.setdefault('attributes', features)
data.setdefault('description', '')
data.setdefault('relation', 'clickbait_sample')
data.setdefault('data', [])
for i in id_labels:
    tmp = [_ for _ in id_features[i]]
    tmp.append(str(id_labels[i]))
    data['data'].append(tmp)

with open('sample_train.arff', 'w') as f:
    f.write(arff.dumps(data))